### Use Concurreny HTTP Requests on Spark
In this example we will use the HTTP Transformer to call a public webAPI.

In [2]:
import json
from requests import Request
from mmlspark.io.http import HTTPTransformer, http_udf
from pyspark.sql.functions import udf, col

# Use any requests from the python requests library
def world_bank_request(country):
  return Request("GET", "http://api.worldbank.org/v2/country/{}?format=json".format(country))

# Create a dataframe with spcificies which countries we want data on
df = (spark.createDataFrame([("br",),("usa",)], ["country"])
  .withColumn("request", http_udf(world_bank_request)(col("country"))))

# Much faster for big data because of the concurrency :)
client = (HTTPTransformer()
      .setConcurrency(3)
      .setInputCol("request")
      .setOutputCol("response"))

# Get the body of the response
def get_response_body(resp):
  return resp.entity.content.decode()
display(client.transform(df).select("country", udf(get_response_body)(col("response")).alias("response")))

StatementMeta(SamplePool, 34, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, 45fbdbd0-a4c7-4a51-ba92-bdeb2562dee9)

In [3]:
spark.stop()

StatementMeta(SamplePool, 34, 3, Finished, Available)